In [ ]:
# Install Lucid

!pip install --quiet lucid==0.2.3

     |████████████████████████████████| 40kB 3.8MB/s 


In [ ]:
%tensorflow_version 1.x
import numpy as np
import tensorflow as tf
import pdb

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform

TensorFlow 1.x selected.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dog2_no_nose.jpg to dog2_no_nose.jpg


In [ ]:
#compare neurons that highly activate for mouth for image vs occluded image

from lucid.misc.io import show, load
from lucid.misc.io.showing import _image_url

googlenet = models.InceptionV1()
googlenet.load_graphdef()


def output_top_neurons(layer, img_url):
    img = load(img_url)
    
    # Compute the activations
    with tf.Graph().as_default(), tf.Session():
        t_input = tf.placeholder(tf.float32, [224, 224, 3]) #empty tensor of fixed size
        T = render.import_model(googlenet, t_input, t_input)
        acts = T(layer).eval({t_input: img})[0]

        top_actv = [[[{"n": n, "v": float(act_vec[n])} for n in np.argsort(-act_vec)[:4]] for act_vec in act_slice] for act_slice in acts]
        flat_list = [item for sublist in top_actv for item in sublist]
    
    top_neurons = []  #contains a list of top 4 neurons for each of 196 patches
    for i in range(196):
      top_neurons_in_patch = []
      for dt in flat_list[i]:  #dt is (neuron#, actv val)
          top_neurons_in_patch.append(list(dt.values()))  #rmv [0] to get both neuron# and its activ val
      top_neurons.append(top_neurons_in_patch)
    return top_neurons
  
def get_neuron_actv(row_num, col_num, neuron_actvs):
  patch_num = 14 * row_num + col_num  #Indices starting at 0
  return neuron_actvs[patch_num]

def get_coords(patch_num):
  return patch_num//14, patch_num % 14   #Indices starting at 0

In [ ]:
def compare_occluded(orig, occluded):
  num_one_common = 0
  num_two_common = 0
  num_three_common = 0
  num_four_common = 0
  for i in range(196):
    x = [q[0] for q in orig[i]] 
    y = [q[0] for q in occluded[i]] 
    common_elem = set(x) - (set(x) - set(y))  #num common neurons in both patches
    if len(common_elem) == 1:
      num_one_common += 1
    elif len(common_elem) == 2:
      num_two_common += 1
    elif len(common_elem) == 3:
      num_three_common += 1
    elif len(common_elem) == 4:
      num_four_common += 1
    elif len(common_elem) == 0:
      print(i, get_coords(i))
  print('# patches w/ 1 neuron in common: ', num_one_common)
  print('# patches w/ 2 neurons in common: ', num_two_common)
  print('# patches w/ 3 neurons in common: ', num_three_common)
  print('# patches w/ 4 neurons in common: ', num_four_common)

In [ ]:
dog_nose = output_top_neurons("mixed4d", 'dog2.jpg')
dog_no_nose = output_top_neurons("mixed4d", 'dog2_no_nose.jpg')
compare_occluded(dog_nose, dog_no_nose)

#using draw_grid_lines_img.py, we manually find that mouth patch is at (9,5) to (11,7)
mouth_actv = get_neuron_actv(9, 6, dog_nose)
print(mouth_actv)

mouth_actv_no_nose = get_neuron_actv(9, 6, dog_no_nose)
print(mouth_actv_no_nose)

76 (5, 6)
89 (6, 5)
90 (6, 6)
103 (7, 5)
104 (7, 6)
132 (9, 6)
# patches w/ 1 neuron in common:  9
# patches w/ 2 neurons in common:  31
# patches w/ 3 neurons in common:  101
# patches w/ 4 neurons in common:  49
[[69, 284.9299011230469], [422, 274.1295166015625], [408, 240.4724884033203], [448, 237.06101989746094]]
[[419, 348.39178466796875], [424, 318.1588134765625], [399, 285.99822998046875], [461, 276.80340576171875]]


In [ ]:
dog_no_mouth = output_top_neurons("mixed4d", 'dog2_no_mouth.jpg')
compare_occluded(dog_nose, dog_no_mouth)

117 (8, 5)
118 (8, 6)
131 (9, 5)
132 (9, 6)
133 (9, 7)
145 (10, 5)
146 (10, 6)
159 (11, 5)
160 (11, 6)
174 (12, 6)
175 (12, 7)
188 (13, 6)
189 (13, 7)
# patches w/ 1 neuron in common:  11
# patches w/ 2 neurons in common:  31
# patches w/ 3 neurons in common:  89
# patches w/ 4 neurons in common:  52


In [ ]:
dog3_nose = output_top_neurons("mixed4d", 'dog3.jpg')
dog3_no_nose = output_top_neurons("mixed4d", 'dog3_no_nose.jpg')
compare_occluded(dog3_nose, dog3_no_nose)

# patches w/ 1 neuron in common:  8
# patches w/ 2 neurons in common:  21
# patches w/ 3 neurons in common:  89
# patches w/ 4 neurons in common:  78


In [ ]:
for i in range(196):
  x = [q[0] for q in dog3_nose[i]] 
  y = [q[0] for q in dog3_no_nose[i]] 
  common_elem = set(x) - (set(x) - set(y))  #num common neurons in both patches
  if len(common_elem) == 1:
    print(i, get_coords(i))

102 (7, 4)
103 (7, 5)
104 (7, 6)
105 (7, 7)
106 (7, 8)
117 (8, 5)
118 (8, 6)
119 (8, 7)
